In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import gym
import conrl as crl
#from tqdm.notebook import tqdm
from tqdm.auto import tqdm

2021-10-26 15:52:00.836216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-26 15:52:00.836274: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import conrl as crl
from conrl import RLmodel, Q

In [15]:
env = gym.make('CartPole-v0').unwrapped

In [16]:
env.action_space

Discrete(2)

In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        # self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        # self.bn1 = nn.BatchNorm2d(16)
        # self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        # self.bn2 = nn.BatchNorm2d(32)
        # self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        # self.bn3 = nn.BatchNorm2d(32)

        # # Number of Linear input connections depends on output of conv2d layers
        # # and therefore the input image size, so compute it.
        # def conv2d_size_out(size, kernel_size = 5, stride = 2):
        #     return (size - (kernel_size - 1) - 1) // stride  + 1
        # convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(8)))
        # convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(8)))
        # linear_input_size = convw * convh * 32
        # self.head = nn.Linear(linear_input_size, 2)
        self.l1 = nn.Linear(4, 24)
        self.l2 = nn.Linear(24,48)
        #self.l3 = nn.Linear(48,8)
        self.l4 = nn.Linear(48,2)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        # x = x.to(device)
        # x = F.relu(self.bn1(self.conv1(x)))
        # x = F.relu(self.bn2(self.conv2(x)))
        # x = F.relu(self.bn3(self.conv3(x)))
        #x = x.to(device)
        x = F.tanh(self.l1(x)) #F.relu(self.l1(x))
        x = F.tanh(self.l2(x)) #F.relu(self.l2(x))
        #x = F.relu(self.l3(x))
        return self.l4(x)
        #self.head(x.view(x.size(0), -1))




In [65]:
import os, wandb
os.environ["WANDB_API_KEY"] = "29ef10a563126c993b31f306e5a17f55b237e430"
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [66]:

logger = crl.logger.WandbRL
logger_config = {
    "project":"test"
}

env_mon = crl.wrapper.MonitorStats(env, 
                        logger=logger, **logger_config)

episode/avg_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode/avg_step_duration_ms,▅▅▃▃▃▃▆▄▄▃▃▂▄▇▁▁▂▃▂▃▃▃▂▇▄▄▄▃▂▂▅▂▂▂▂▅▅█▂▂
episode/episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
episode/return,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode/smooth_return,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode/steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode/avg_reward,1.0
episode/avg_step_duration_ms,0.28396
episode/episode,999
episode/return,1.0
episode/smooth_return,1.0


2021-10-26 16:21:00.957058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-26 16:21:00.957125: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [67]:
m = DQN()
q = Q(env_mon, m, torch.optim.Adam)
q_targ = q.copy()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [68]:
pi = crl.explore.EpsilonGreedy(epsilon=1, q=q)

In [69]:
tape = crl.recorder.NStepRecorder(n=1, gamma=1)
buffer = crl.replay.ReplayBuffer(capacity=100000)

qlearning = crl.objective.QLearning(q, q_targ=q, 
                                    loss_function=F.mse_loss)
                                

In [70]:
episodes = 1000
for _ in tqdm(range(episodes), desc="Episode"):
    
    s = env_mon.reset()

    for t in range(env_mon.spec.max_episode_steps):
        a = pi.act(s)
        s_next,r,done,_ = env_mon.step(a)

        tape.add(s,a,r,done,s_next)
        while tape:
            transition = tape.pop()
        buffer.add(transition)

        s = s_next

        if done:
            break

        if len(buffer) >= 64:
            #break
            transition_batch = buffer.sample(batch_size=64)
            qlearning.update(transition_batch)

            pi.update(decay=0.995)

Episode:   0%|          | 0/1000 [00:00<?, ?it/s]

Episode: 1 | Timesteps: 12 |                    Avg_reward: 1.0 | Returns: 12.0
Episode: 2 | Timesteps: 16 |                    Avg_reward: 1.0 | Returns: 16.0
Episode: 3 | Timesteps: 19 |                    Avg_reward: 1.0 | Returns: 19.0
Episode: 4 | Timesteps: 21 |                    Avg_reward: 1.0 | Returns: 21.0
Episode: 5 | Timesteps: 19 |                    Avg_reward: 1.0 | Returns: 19.0
Episode: 6 | Timesteps: 15 |                    Avg_reward: 1.0 | Returns: 15.0
Episode: 7 | Timesteps: 31 |                    Avg_reward: 1.0 | Returns: 31.0
Episode: 8 | Timesteps: 13 |                    Avg_reward: 1.0 | Returns: 13.0
Episode: 9 | Timesteps: 19 |                    Avg_reward: 1.0 | Returns: 19.0
Episode: 10 | Timesteps: 10 |                    Avg_reward: 1.0 | Returns: 10.0
Episode: 11 | Timesteps: 15 |                    Avg_reward: 1.0 | Returns: 15.0
Episode: 12 | Timesteps: 18 |                    Avg_reward: 1.0 | Returns: 18.0
Episode: 13 | Timesteps: 10 |        

In [71]:
pi.epsilon

0.00998645168764533

In [60]:
tb = buffer.sample()

qpar = list(q.parameters())
qpar_targ = list(q_targ.parameters())


In [61]:
qpar[0], qpar_targ[0]

(Parameter containing:
 tensor([[ 1.0746e-01,  2.4111e-01, -1.8648e-01, -4.8257e-01],
         [ 1.4174e-01, -4.8032e-01,  4.9222e-01, -3.1432e-01],
         [ 2.7982e-01,  1.1091e-02,  2.3578e-01,  4.1365e-02],
         [-4.1854e-01,  2.9072e-01, -1.0693e-01,  2.4301e-01],
         [ 3.1353e-01, -3.6036e-01,  2.7875e-01,  1.7666e-01],
         [-4.7428e-01,  1.9394e-02, -3.2474e-01,  1.3701e-01],
         [ 2.0070e-01,  4.4073e-01,  4.7930e-01, -3.8833e-01],
         [ 3.7280e-01, -4.4066e-01,  8.8306e-02, -3.2490e-02],
         [-3.3622e-01,  3.4281e-02,  2.0605e-01, -1.1733e-01],
         [ 4.3834e-01, -2.8639e-01, -3.3712e-01,  4.1877e-01],
         [ 1.3006e-01,  4.9945e-01,  1.5341e-01, -1.5578e-01],
         [-3.5822e-01,  9.9626e-02,  4.7989e-01,  2.2184e-02],
         [ 3.2841e-01,  2.2090e-01,  2.3514e-04,  3.5021e-01],
         [ 3.6171e-01, -1.0413e-01, -3.1198e-01, -4.5960e-01],
         [-1.0440e-02, -3.3329e-01, -2.1590e-01,  1.8520e-01],
         [-3.9443e-01,  4.2242e-

In [62]:
qlearning.update(tb)

In [63]:
qpar[0], qpar_targ[0]

(Parameter containing:
 tensor([[ 0.1085,  0.2401, -0.1875, -0.4816],
         [ 0.1427, -0.4793,  0.4932, -0.3153],
         [ 0.2808,  0.0101,  0.2348,  0.0424],
         [-0.4175,  0.2897, -0.1079,  0.2440],
         [ 0.3125, -0.3594,  0.2797,  0.1757],
         [-0.4733,  0.0184, -0.3257,  0.1380],
         [ 0.2017,  0.4397,  0.4783, -0.3873],
         [ 0.3718, -0.4397,  0.0893, -0.0335],
         [-0.3372,  0.0333,  0.2050, -0.1163],
         [ 0.4393, -0.2854, -0.3361,  0.4178],
         [ 0.1291,  0.5005,  0.1544, -0.1568],
         [-0.3592,  0.1006,  0.4809,  0.0212],
         [ 0.3274,  0.2219,  0.0012,  0.3492],
         [ 0.3627, -0.1051, -0.3130, -0.4586],
         [-0.0114, -0.3323, -0.2149,  0.1842],
         [-0.3934,  0.4234,  0.3748,  0.4908],
         [ 0.0979,  0.1370,  0.2192,  0.0325],
         [ 0.3203, -0.0393,  0.1293, -0.3647],
         [-0.3517,  0.0997, -0.0478,  0.2022],
         [ 0.3084, -0.4000,  0.1588, -0.3680],
         [ 0.0489, -0.3460,  0.0778, 

In [ ]:
episodes = 1000
for _ in tqdm(range(episodes), desc="Episode"):
    s = env_mon.reset()
    #pi.epsilon *= 0.90
    done = False
    #for t in range(env_mon.spec.max_episode_steps):
    while not done:
        a = pi.act(s)
        s_next,r,done,_ = env_mon.step(a)

        tape.add(s,a,r,done,s_next)
        while tape:
            transition = tape.pop()
        buffer.add(transition)

        if len(buffer) > 100:
            transition_batch = buffer.sample(batch_size=64)
            qlearning.update(transition_batch)
            #q_targ.soft_update(q, 0.9)
            

        if done: break

        s = s_next

In [ ]:
from matplotlib import animation
import matplotlib.pyplot as plt

def save_frames_as_gif(frames, filename='gym_animation.gif'):

    #Mess with this to change frame size
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)

    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    anim.save(filename, writer='imagemagick', fps=60)


observation = env.reset()
frames = []
actions = []
rewards = []
for t in range(1000):
    #Render to frames buffer
    frames.append(env.render(mode="rgb_array"))
    action = pi.act(observation, explore=False)
    actions.append(action)
    observation, r, done, _ = env.step(action)
    rewards.append(r)
    if done:
        break
env.close()
#save_frames_as_gif(frames)

In [ ]:
env.action_space.shape

In [ ]:
from collections import deque
a = deque(maxlen=3)

a.append(1)

In [ ]:
len(rewards) == sum(rewards)

In [ ]:
a.append(4)

In [ ]:
a

In [ ]:
np.mean(a)

In [ ]:
import tensorflow as tf